In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

In [2]:
number_of_classes = 4 
batch_size = 16
epochs = 30 
file_path_train = '/Users/borak/Grad/archive/training'
file_path_test = '/Users/borak/Grad/archive/testing'
train_datagenetor = ImageDataGenerator()
test_datagenetor = ImageDataGenerator()

In [3]:
train_data = train_datagenetor.flow_from_directory(
    file_path_train,
    target_size=(224, 224), 
    batch_size=batch_size,
    class_mode='categorical',
)

test_data = test_datagenetor.flow_from_directory(
    file_path_test,
    target_size=(224, 224),
    shuffle = False,
    batch_size=batch_size,
    class_mode='categorical',
)

base_model = VGG16(input_shape=(224,224,3),
                   include_top = False,
                   weights = 'imagenet')

Found 5712 images belonging to 4 classes.
Found 1311 images belonging to 4 classes.


58889256/58889256 [==============================] - 11s 0us/step


In [4]:
for layer in base_model.layers:
    layer.trainable = False
    
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(4096, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)
x = layers.Dense(4096, activation = 'relu')(x)
x = layers.Dropout(0.5)(x)

predictions = layers.Dense(number_of_classes, activation = 'softmax')(x)
model = models.Model(inputs = base_model.input, outputs = predictions)

In [ ]:
model.compile(optimizer = 'adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
history = model.fit(train_data, epochs = epochs)
score = model.evaluate(test_data)
print('Test accuracy :', score[1])


Epoch 1/30


357/357 [==============================] - 939s 3s/step - loss: 19.8646 - accuracy: 0.7964
Epoch 2/30
357/357 [==============================] - 927s 3s/step - loss: 3.5771 - accuracy: 0.8696
Epoch 3/30
357/357 [==============================] - 938s 3s/step - loss: 3.2595 - accuracy: 0.8831
Epoch 4/30
357/357 [==============================] - 951s 3s/step - loss: 2.7461 - accuracy: 0.8992
Epoch 5/30
357/357 [==============================] - 1012s 3s/step - loss: 2.4400 - accuracy: 0.9112
Epoch 6/30
357/357 [==============================] - 1046s 3s/step - loss: 4.1450 - accuracy: 0.9025
Epoch 7/30
306/357 [========================>.....] - ETA: 2:15 - loss: 8.2840 - accuracy: 0.8889

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 4))
plt.subplot(1, 2, 1)
plt.plot(train_loss, label='Train Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss over Epochs')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(train_accuracy, label='Train Accuracy', color='orange')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training Accuracy over Epochs')
plt.legend()

y_true = test_data.classes
y_pred_classes = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_true, y_pred_classes)
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

conf_matrix = confusion_matrix(y_true, y_pred_classes)

plt.figure(figsize=(8, 6))

sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=train_data.class_indices, yticklabels=train_data.class_indices)
plt.title('Confusion Matrix')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

print('Accuracy:', accuracy)
print('Precision:', precision)
print('Recall:', recall)
print('F1-Score:', f1)

In [ ]:
model.save('VGG_NET_Model_Result2.h5')